In [ ]:

import sim_generator as sg
import pandas as pd


feature_values = {
    'yesterday_total_packages': 8030,
    'RAFT_known_shipped_pkg_count': 2186,
    'RAFT_predicted_carryover_pkg_count': 109,
    'RAFT_predicted_total_handoff_pkg_count': 5691,
    'Day of Week_Sunday': 0,
    'Day of Week_Monday': 0,
    'Day of Week_Tuesday': 0,
    'Day of Week_Wednesday': 0,
    'Day of Week_Thursday': 1,
    'Day of Week_Friday': 0,
    'Day of Week_Saturday': 0,
    'Promotion': 0,
    'TMAX': 52.8,
    'TMIN': 32.8,
    'AWND': 9.1,
    'PRCP': 0,
    'SNOW': 0
        }


In [14]:
results_tracker = {}
for i in range(0, 100):

    df_pallets, df_package_distribution, TFC_arrival_minutes = sg.simulation_generator(False, feature_values)

    day_pallets = df_pallets.groupby('Pallet').agg(
        num_packages=('package_tracking_number', 'count'),
        earliest_arrival=('pkg_received_utc_ts', 'min'),
        packages=('package_tracking_number', lambda x: list(zip(x, df_pallets.loc[x.index, 'scac']))),
        linehaul=('Linehaul', 'first')
    ).reset_index()

    def sum_packages_by_linehaul(df, linehaul):
        filtered_df = df[df['linehaul'] == linehaul]
        all_lh_packages = sum([len(row['packages']) for i, row in filtered_df.iterrows()])
        return all_lh_packages

    total_packages = sum([len(row['packages']) for i, row in day_pallets.iterrows()])
    all_packages = [pkg for sublist in day_pallets['packages'] for pkg in sublist]
    total_packages_NC = len([pkg for pkg in all_packages if pkg[1] in ['UPSN', 'USPS', 'FDEG', 'FDE']])
    total_packages_UPSN = len([pkg for pkg in all_packages if pkg[1] in ['UPSN']])
    total_packages_USPS = len([pkg for pkg in all_packages if pkg[1] in ['USPS']])
    total_packages_FDEG = len([pkg for pkg in all_packages if pkg[1] in ['FDEG']])
    total_packages_FDE = len([pkg for pkg in all_packages if pkg[1] in ['FDE']])
    linehaul_A_packages = sum_packages_by_linehaul(day_pallets, 'A')
    linehaul_B_packages = sum_packages_by_linehaul(day_pallets, 'B')
    linehaul_C_packages = sum_packages_by_linehaul(day_pallets, 'C')
    linehaul_TFC_packages = sum_packages_by_linehaul(day_pallets, 'TFC')

    def filter_packages_by_linehaul(df, linehaul):
        return df[df['linehaul'] == linehaul]

    def sum_packages_by_type(filtered_df, package_type):
        all_packages = [pkg for sublist in filtered_df['packages'] for pkg in sublist]
        target_packages = len([pkg for pkg in all_packages if pkg[1] in [package_type]])
        return target_packages


    # Filter DataFrames by linehaul
    linehaul_A_df = filter_packages_by_linehaul(day_pallets, 'A')
    linehaul_B_df = filter_packages_by_linehaul(day_pallets, 'B')
    linehaul_C_df = filter_packages_by_linehaul(day_pallets, 'C')
    linehaul_TFC_df = filter_packages_by_linehaul(day_pallets, 'TFC')
    total_packages_TLMD = total_packages - total_packages_NC

    results = {
    # Total Packages
    "TOTAL_PACKAGES": total_packages,
    "TOTAL_PACKAGES_TLMD": total_packages_TLMD,
    "TOTAL_PACKAGES_NC": total_packages_NC,
    "TOTAL_PACKAGES_TFC": linehaul_TFC_packages}

    results_tracker[i] = results

df = pd.DataFrame.from_dict(results_tracker, orient='index')

c:\Users\fenst\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\fenst\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\fenst\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to un